In [1]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class neuralNetwork:
    def __init__(self, inputNodes, hiddenNodes_1, hiddenNodes_2, outputNodes, learningGrate):
        self.inodes = inputNodes
        self.h1nodes = hiddenNodes_1
        self.h2nodes = hiddenNodes_2
        self.onodes = outputNodes

        self.lr = learningGrate

        # wij代表i结点到下一层的j结点的权重

        # 简单初始化权重
        # self.wih = np.random.rand(self.hnodes, self.inodes) - 0.5
        # self.who = np.random.rand(self.onodes, self.hnodes) - 0.5

        # 复杂的初始化权重，使用均值为0，方差为"1/传入链接数"的正态分布采样
        # 传入链接数应该是前一层的结点数，第二个参数书上是本层结点数，我改成上一层结点数
        self.wih1 = np.random.normal(0.0, self.inodes ** -0.5, (self.h1nodes, self.inodes))
        self.wh1h2 = np.random.normal(0.0, self.h1nodes ** -0.5, (self.h2nodes, self.h1nodes))
        self.wh2o = np.random.normal(0.0, self.h2nodes ** -0.5, (self.onodes, self.h2nodes))

        self.activation_function = scipy.special.expit

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        hidden1_inputs = np.dot(self.wih1, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)

        hidden2_inputs = np.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)

        final_inputs = np.dot(self.wh2o, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)

        output_errors = targets - final_outputs
        hidden2_errors = np.dot(self.wh2o.T, output_errors)
        hidden1_errors = np.dot(self.wh1h2.T, hidden2_errors)

        # 调整隐藏层和输出层之间的权重
        self.wh2o += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), hidden2_outputs.T)
        # 调整隐藏层1和隐藏层2之间的权重
        self.wh1h2 += self.lr * np.dot((hidden2_errors * hidden2_outputs * (1.0 - hidden2_outputs)), hidden1_outputs.T)
        # 调整输入层与隐藏层时间的权重
        self.wih1 += self.lr * np.dot((hidden1_errors * hidden1_outputs * (1.0 - hidden1_outputs)), inputs.T)

    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T

        hidden1_inputs = np.dot(self.wih1, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)

        hidden2_inputs = np.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)

        final_inputs = np.dot(self.wh2o, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

In [3]:
# 神经网络实例训练识别手写数字
input_nodes = 784
hidden1_nodes = 100
hidden2_nodes = 100
output_nodes = 10

learning_grate = 0.2

# 实例化训练器
n = neuralNetwork(input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_grate)
# 读入训练数据
with open('mnist/mnist_train.csv', 'r') as training_data_file:
    training_data_list = training_data_file.readlines()
# 训练
from time import perf_counter as pc
start = pc()
for _ in range(2):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = np.asfarray(all_values[1:]) / 255.0 * 0.99 + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

print(pc() - start)


# 测试识别结果
with open('mnist/mnist_test.csv', 'r') as test_data_file:
    test_data_list = test_data_file.readlines()
# 逐行测试
result = []
score = 0
start = pc()
for record in test_data_list:
    label = int(record[0])
    all_values = record[1:].split(',')
    res = n.query(np.asfarray(all_values[1:]) / 255.0 * 0.99 + 0.01)
    answer = res.argmax()
    # result.append((label, answer))
    if label == answer:
        score += 1

print(pc() - start)
# print(result)
print(score / 10000)

58.87840628279341
2.2673896731096903
0.842
